In [29]:
!pip install rdkit

In [30]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Descriptors

# =================================================================
# 1. CONFIGURACIÓN DEL PROCESO (INPUTS)
# =================================================================
ACEITE_BASE_KG = 100.0   # Base de cálculo
RATIO_MOLAR = 6.0        # Metanol : Aceite
KOH_PCT_PESO = 0.8       # % KOH respecto al aceite
TEMPERATURA_C = 60.0     # Temperatura de reacción
CONVERSION_RXN = 0.985   # 98.5% de conversión (típico industrial)

# SMILES de las especies
SMILES_SPECIES = {
    'Aceite_Palma': 'CCCCCCCC=CCCCCCCCC(=O)OCC(OC(=O)CCCCCCCC=CCCCCCCCC)COC(=O)CCCCCCCC=CCCCCCCCC',
    'Metanol': 'CO',
    'Biodiesel_FAME': 'CCCCCCCC=CCCCCCCCC(=O)OC',
    'Glicerina': 'OCC(O)CO',
    'KOH': '[K+].[OH-]'
}

def ejecutar_simulacion():
    # =================================================================
    # 2. CÁLCULOS QUÍMICOS (RDKit)
    # =================================================================
    # Pesos Moleculares (g/mol)
    mw = {k: Descriptors.MolWt(Chem.MolFromSmiles(v)) for k, v in SMILES_SPECIES.items()}

    # Moles de entrada
    n_aceite = (ACEITE_BASE_KG * 1000) / mw['Aceite_Palma']
    n_meoh_necesario = n_aceite * RATIO_MOLAR

    # Masas de entrada (kg)
    m_meoh_total = (n_meoh_necesario * mw['Metanol']) / 1000
    m_koh = ACEITE_BASE_KG * (KOH_PCT_PESO / 100)
    m_total_entrada = ACEITE_BASE_KG + m_meoh_total + m_koh

    # =================================================================
    # 3. ESTEQUIOMETRÍA DE LA REACCIÓN
    # 1 Aceite + 3 MeOH -> 3 FAME + 1 Glicerina
    # =================================================================
    n_reaccionado = n_aceite * CONVERSION_RXN

    # Producción
    n_fame_prod = n_reaccionado * 3
    n_glyc_prod = n_reaccionado

    # Masas de salida (kg)
    m_fame = (n_fame_prod * mw['Biodiesel_FAME']) / 1000
    m_glyc = (n_glyc_prod * mw['Glicerina']) / 1000

    # Sobrantes y residuos
    n_meoh_residual = n_meoh_necesario - (3 * n_reaccionado)
    m_meoh_res = (n_meoh_residual * mw['Metanol']) / 1000
    m_aceite_res = (n_aceite - n_reaccionado) * mw['Aceite_Palma'] / 1000

    m_total_salida = m_fame + m_glyc + m_meoh_res + m_aceite_res + m_koh

    # =================================================================
    # 4. REPORTE DE RESULTADOS (VISIBLES)
    # =================================================================
    print("="*60)
    print("      REPORTE TÉCNICO: SÍNTESIS DE BIODIÉSEL (PALMA)")
    print("="*60)

    print(f"\n[ CONDICIONES DE OPERACIÓN ]")
    print(f"Temperatura: {TEMPERATURA_C} °C")
    print(f"Relación Molar (MeOH:Aceite): {RATIO_MOLAR}:1")
    print(f"Catalizador (KOH): {KOH_PCT_PESO}% en peso")
    print(f"Conversión alcanzada: {CONVERSION_RXN*100:.2f}%")

    print(f"\n[ BALANCE DE MATERIA GLOBAL ]")
    print(f"{'Corriente':<20} | {'Masa Entrada (kg)':<18} | {'Masa Salida (kg)':<18}")
    print("-" * 60)
    print(f"{'Aceite de Palma':<20} | {ACEITE_BASE_KG:<18.3f} | {m_aceite_res:<18.3f}")
    print(f"{'Metanol':<20} | {m_meoh_total:<18.3f} | {m_meoh_res:<18.3f}")
    print(f"{'Catalizador KOH':<20} | {m_koh:<18.3f} | {m_koh:<18.3f}")
    print(f"{'Biodiésel (FAME)':<20} | {'0.000':<18} | {m_fame:<18.3f}")
    print(f"{'Glicerina':<20} | {'0.000':<18} | {m_glyc:<18.3f}")
    print("-" * 60)
    print(f"{'TOTAL':<20} | {m_total_entrada:<18.3f} | {m_total_salida:<18.3f}")

    # Verificación de error de balance
    error = abs(m_total_entrada - m_total_salida)
    print(f"Error de balance de masa: {error:.2e} kg")

    print(f"\n[ INDICADORES DE RENDIMIENTO (KPIs) ]")
    rendimiento_masa = (m_fame / ACEITE_BASE_KG) * 100
    print(f"Rendimiento de Biodiésel (kg FAME / 100kg Aceite): {rendimiento_masa:.2f} %")
    print(f"Pureza estimada de la fase orgánica: {(m_fame/(m_fame+m_aceite_res+m_meoh_res*0.2))*100:.2f} %")

    # =================================================================
    # 5. FRAGMENTACIÓN MOLECULAR (VISUALIZACIÓN)
    # =================================================================
    print(f"\n[ ANÁLISIS DE GRUPOS FUNCIONALES (FRAGMENTACIÓN) ]")
    mol = Chem.MolFromSmiles(SMILES_SPECIES['Aceite_Palma'])
    # Contamos grupos CH2 (cadena alifática) y COO (éster)
    n_ch2 = len(mol.GetSubstructMatches(Chem.MolFromSmarts("[CH2]")))
    n_ester = len(mol.GetSubstructMatches(Chem.MolFromSmarts("C(=O)O")))
    print(f"Molécula: Trioleína (Aceite)")
    print(f" - Grupos CH2 (Metilenos): {n_ch2}")
    print(f" - Grupos C=O (Ésteres): {n_ester} (Puntos de reacción)")

In [31]:
ejecutar_simulacion()

      REPORTE TÉCNICO: SÍNTESIS DE BIODIÉSEL (PALMA)

[ CONDICIONES DE OPERACIÓN ]
Temperatura: 60.0 °C
Relación Molar (MeOH:Aceite): 6.0:1
Catalizador (KOH): 0.8% en peso
Conversión alcanzada: 98.50%

[ BALANCE DE MATERIA GLOBAL ]
Corriente            | Masa Entrada (kg)  | Masa Salida (kg)  
------------------------------------------------------------
Aceite de Palma      | 100.000            | 1.500             
Metanol              | 22.062             | 11.196            
Catalizador KOH      | 0.800              | 0.800             
Biodiésel (FAME)     | 0.000              | 95.785            
Glicerina            | 0.000              | 10.410            
------------------------------------------------------------
TOTAL                | 122.862            | 119.691           
Error de balance de masa: 3.17e+00 kg

[ INDICADORES DE RENDIMIENTO (KPIs) ]
Rendimiento de Biodiésel (kg FAME / 100kg Aceite): 95.78 %
Pureza estimada de la fase orgánica: 96.24 %

[ ANÁLISIS DE GRUPOS FU